
### Logs
* 30.08: use 2048 for max_len and max_position_embeddings
* 03.09: use 512 instead
* 23.09: 512 pretrained with both datasets in MLM. training with 2048 and try to infer with 512. Pretraining notebook: pretrain1_task_ver3(512_larger dataset)
* 25.09 512 pretrained with both datasets in MLM. training with 1536. Pretraining notebook: pretrain1_task_ver3(512_larger dataset)
* no pretraining
* no pretraining use max_length as 1600, learning rate change to 2e-5
* no pretraining use max_length as 1600, learning rate changed to 4e-6, cosine scheduler
* no pretraining use max_legnth as 1600, learning rate changed back to 2e-6, cosine scheduler
* change weight decay from 0.01 to 1e-4, learning rate changed to 4e-6
* pretraining 1009, weight decay 1e-4, learning reate 4e-6
* pretraining 2109, weight decay 1e-4, learning reate 4e-6
* batch_size 8, eval batch_size 16
* use pretrained model on new generated data

In [1]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install datasets -q
!pip install accelerate -U -q

In [2]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

from datasets import Dataset, disable_progress_bar
import pandas as pd

pdf = pd.read_csv("./input/prompts_train.csv")
sdf = pd.read_csv("./input/summaries_train.csv")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /microsoft/deberta-v3-large/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f5a9a76abb0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 4660ee25-d350-4374-a7c0-64e5f4a83a7d)')' thrown while requesting HEAD https://huggingface.co/microsoft/deberta-v3-large/resolve/main/tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have

In [3]:
from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large")
## use the pretrained model
# from transformers import AutoConfig
# config = AutoConfig.from_pretrained('./input/pretrain/pretrained_model/')
# model = AutoModelForSequenceClassification.from_pretrained('./input/pretrain/pretrained_model', config = config)

## Train

In [4]:
%%writefile train.py

import os
import logging
import warnings
from dataclasses import dataclass, field
from typing import Optional

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    set_seed,
    Trainer,
    TrainingArguments,
    HfArgumentParser,
    DataCollatorWithPadding,
)
from datasets import Dataset, disable_progress_bar
import pandas as pd
import numpy as np

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['WANDB_PROJECT'] = 'kaggle-commonlit-eval-student-summaries-0910-pretrain-new-data'

disable_progress_bar()

@dataclass
class Config:
    model_name_or_path: Optional[str] = field(
        default="microsoft/deberta-v3-large",
        metadata={"help": "Model name or path"},
    )

    data_dir: Optional[str] = field(
        default="/kaggle/input/commonlit-evaluate-student-summaries",
        metadata={"help": "Data directory"},
    )

    max_seq_length: Optional[int] = field(
        default=1600,
        #default = 512,
        #default = 512,
        metadata={"help": "Max sequence length"},
    )

    add_prompt_question: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt question into input"},
    )

    add_prompt_text: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt text into input"},
    )

    fold: Optional[int] = field(
        default=0,
        metadata={"help": "Fold"},
    )

    num_proc: Optional[int] = field(
        default=4,
        metadata={"help": "Number of processes"},
    )

    dropout: Optional[float] = field(
        default=0.,
        metadata={"help": "Amount of dropout to apply"},
    )
    max_position_embeddings: Optional[int] = field(
        default=1600,
        #default=512,
        #default = 512,
        metadata={"help": "Amount of dropout to apply"},
    )


# Spell auto correction
# from spellchecker import SpellChecker

# def correct_spelling(input_text):
#     print('input_text: ', input_text)
#     # Initialize the spell checker
#     spell = SpellChecker()
#     # Split the input text into words
#     words = input_text.split()
#     # Initialize an empty list to store the corrected words
#     corrected_words = []
#     for word in words:
#         # Check if the word has any punctuation at the end
#         if word[-1].isalpha():
#             # Extract the punctuation
#             punctuation_end = ""
#         else:
#             punctuation_end = word[-1]
#             word = word[:-1]
#         # check if the word has any punctuation at the start
#         if word[0].isalpha():
#            # Extract the punctuation
#             punctuation_start = ""
#         else:
#             punctuation_start = word[0]
#             word = word[1:]
#         # Check the spelling of the word (case insensitive)
#         corrected_word = spell.correction(word.lower())
#         # Preserve the original capitalization
#         if word[0].isupper():
#             corrected_word = corrected_word.capitalize()
#         # Combine the corrected word and punctuation (if any)
#         corrected_word = punctuation_start+corrected_word+punctuation_end
#         # Append the corrected word to the list
#         corrected_words.append(corrected_word)
#     # Join the corrected words back into a single string
#     corrected_text = " ".join(corrected_words)
#     return corrected_text

def tokenize(example, tokenizer, config):
    sep = tokenizer.sep_token

    # if config.add_prompt_question:
    #     text = sep.join(
    #         [example["prompt_question"], example["prompt_text"], example["text"]]
    #     )
    # elif config.add_prompt_text:
    #     text = sep.join([example["prompt_text"], example["text"]])
    # else:
    #     text = example["text"]
    prompt = sep.join([example["prompt_title"], example["prompt_text"], example["prompt_question"]])
    labels = [example["content"], example["wording"]]

    tokenized = tokenizer(
#         prompt,
#         example["text"],
        example['text'],
        prompt,
        padding=False,
        truncation=True, # changed by Peng, turn on the truncation
        max_length=config.max_seq_length,
    )

    return {
        **tokenized,
        "labels": labels,
    }




def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }


def main():
    parser = HfArgumentParser((Config, TrainingArguments))

    config, training_args = parser.parse_args_into_dataclasses()

    set_seed(training_args.seed)

    if "wandb" in training_args.report_to:
        import wandb

        try:
#             from kaggle_secrets import UserSecretsClient
#             user_secrets = UserSecretsClient()
#             key = user_secrets.get_secret("wandb")

#             wandb.login(key=key)
            wandb.login()
        except:
            print("Could not log in to WandB")

    tokenizer = AutoTokenizer.from_pretrained(config.model_name_or_path)
    model_config = AutoConfig.from_pretrained(config.model_name_or_path)

    model_config.update({
        "hidden_dropout_prob": config.dropout,
        "attention_probs_dropout_prob": config.dropout,
        "num_labels": 2,
        "problem_type": "regression",
        "max_position_embeddings": config.max_position_embeddings,
        "cfg": config.__dict__,
    })

    print(model_config)
    
    # Do not use pretrained model
#     print('do not use pretrained model')
#     model = AutoModelForSequenceClassification.from_pretrained(
#         config.model_name_or_path, config=model_config
#     )
    
    # use pretrained model
    print('use pretrained_model')
    #print('./pretrain_seed42_0810/')
    print('./pretrain_seed42_0910/')
    model = AutoModelForSequenceClassification.from_pretrained('./pretrain_seed42_0910/', config = model_config)

    #pdf = pd.read_csv(f"{config.data_dir}/prompts_train.csv")
    pdf = pd.read_csv(f"./input/prompts_train.csv")
    #sdf = pd.read_csv(f"{config.data_dir}/summaries_train.csv")
    sdf = pd.read_csv(f"./input/summaries_train.csv")

    df = pdf.merge(sdf, on="prompt_id")

    # 4 prompt ids, 4 folds
    id2fold = {
        "814d6b": 0,
        "39c16e": 1,
        "3b9047": 2,
        "ebad26": 3,
    }

    df["fold"] = df["prompt_id"].map(id2fold)

    train_ds = Dataset.from_pandas(df[df["fold"] != config.fold])
    val_ds = Dataset.from_pandas(df[df["fold"] == config.fold])

    train_ds = train_ds.map(
        tokenize,
        batched=False,
        num_proc=config.num_proc,
        fn_kwargs={"tokenizer": tokenizer, "config": config},
    )

    val_ds = val_ds.map(
        tokenize,
        batched=False,
        num_proc=config.num_proc,
        fn_kwargs={"tokenizer": tokenizer, "config": config},
    )

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        pad_to_multiple_of=16 if training_args.fp16 else None,
    )

    training_args.bf16 =True
    training_args.gradient_accumulation_steps = 1
    training_args.load_best_model_at_end = True
    training_args.greater_is_better = False
    training_args.metric_for_best_model = 'mcrmse'
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_mcrmse,
    )

    trainer.train()

    model.config.best_metric = trainer.state.best_metric
    model.config.save_pretrained(training_args.output_dir)

    trainer.log({"eval_best_mcrmse": trainer.state.best_metric})


if __name__ == "__main__":
    main()

Overwriting train.py


In [5]:
from pathlib import Path

seed = 42

fold = 1

output = f"output_fold{fold}_seed{seed}_1010"

!python train.py \
  --model_name_or_path "microsoft/deberta-v3-large" \
  --add_prompt_question True \
  --fold $fold \
  --data_dir "./" \
  --output_dir $output \
  --fp16 \
  --num_train_epochs 4 \
  --dataloader_num_workers 4 \
  --learning_rate 6e-6 \
  --weight_decay 1e-4 \
  --warmup_ratio 0 \
  --lr_scheduler_type "cosine" \
  --optim "adamw_torch" \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --evaluation_strategy "steps" \
  --eval_steps 150 \
  --save_strategy "steps" \
  --save_steps 150 \
  --save_total_limit 1 \
  --report_to "wandb" \
  --metric_for_best_model "mcrmse" \
  --greater_is_better False \
  --logging_steps 10 \
  --log_level "error" \
  --disable_tqdm True \
  --ddp_find_unused_parameters False \
  --dropout 0 \
  --seed $seed


output_dir = Path.cwd() / output
# add json files
for json_file in output_dir.glob("checkpoint*/*token*.json"):
    json_file.rename(output_dir/json_file.name)

# model files
for model_file in output_dir.glob("checkpoint*/*model*"):
    model_file.rename(output_dir/model_file.name)

# remove optimizer states and other files
to_delete = str(list(output_dir.glob("checkpoint*"))[0])
!rm -r $to_delete

wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_probs_dropout_prob": 0.0,
  "cfg": {
    "add_prompt_question": true,
    "add_prompt_text": false,
    "data_dir": "./",
    "dropout": 0.0,
    "fold": 1,
    "max_position_embeddings": 1600,
    "max_seq_length": 1600,
    "model_name_or_path": "microsoft/deberta-v3-large",
    "num_proc": 4
  },
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 1600,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
 

{'loss': 0.4154, 'learning_rate': 5.9257484435388175e-06, 'epoch': 0.29}
{'loss': 0.2811, 'learning_rate': 5.923694785671475e-06, 'epoch': 0.29}
{'loss': 0.3631, 'learning_rate': 5.921613479425531e-06, 'epoch': 0.29}
{'eval_loss': 0.30140700936317444, 'eval_content_rmse': 0.4005008935928345, 'eval_wording_rmse': 0.6651411652565002, 'eval_mcrmse': 0.5328210592269897, 'eval_runtime': 112.6022, 'eval_samples_per_second': 18.268, 'eval_steps_per_second': 9.138, 'epoch': 0.29}
{'loss': 0.3045, 'learning_rate': 5.919504544483185e-06, 'epoch': 0.3}
{'loss': 0.1667, 'learning_rate': 5.917368000787912e-06, 'epoch': 0.3}
{'loss': 0.283, 'learning_rate': 5.915203868544272e-06, 'epoch': 0.31}
{'loss': 0.4218, 'learning_rate': 5.9130121682177206e-06, 'epoch': 0.31}
{'loss': 0.2482, 'learning_rate': 5.910792920534416e-06, 'epoch': 0.31}
{'loss': 0.3322, 'learning_rate': 5.908546146481025e-06, 'epoch': 0.32}
{'loss': 0.2646, 'learning_rate': 5.906271867304515e-06, 'epoch': 0.32}
{'loss': 0.2921, 'lea

{'eval_loss': 0.3581071197986603, 'eval_content_rmse': 0.3866088390350342, 'eval_wording_rmse': 0.7528263926506042, 'eval_mcrmse': 0.5697176456451416, 'eval_runtime': 113.2292, 'eval_samples_per_second': 18.167, 'eval_steps_per_second': 9.088, 'epoch': 0.65}
{'loss': 0.24, 'learning_rate': 5.619783768318273e-06, 'epoch': 0.65}
{'loss': 0.2613, 'learning_rate': 5.615276238935198e-06, 'epoch': 0.65}
{'loss': 0.2387, 'learning_rate': 5.610743977781915e-06, 'epoch': 0.66}
{'loss': 0.2604, 'learning_rate': 5.6061870277184595e-06, 'epoch': 0.66}
{'loss': 0.3785, 'learning_rate': 5.60160543183835e-06, 'epoch': 0.67}
{'loss': 0.1984, 'learning_rate': 5.596999233468163e-06, 'epoch': 0.67}
{'loss': 0.2117, 'learning_rate': 5.592368476167139e-06, 'epoch': 0.67}
{'loss': 0.3769, 'learning_rate': 5.587713203726757e-06, 'epoch': 0.68}
{'loss': 0.4068, 'learning_rate': 5.583033460170336e-06, 'epoch': 0.68}
{'loss': 0.1522, 'learning_rate': 5.578329289752604e-06, 'epoch': 0.69}
{'loss': 0.347, 'learni

{'loss': 0.257, 'learning_rate': 5.120667901410232e-06, 'epoch': 1.0}
{'loss': 0.1472, 'learning_rate': 5.114132460161414e-06, 'epoch': 1.01}
{'loss': 0.1456, 'learning_rate': 5.107577026287185e-06, 'epoch': 1.01}
{'loss': 0.1698, 'learning_rate': 5.101001661780032e-06, 'epoch': 1.01}
{'loss': 0.1456, 'learning_rate': 5.0944064288209205e-06, 'epoch': 1.02}
{'loss': 0.1878, 'learning_rate': 5.087791389778709e-06, 'epoch': 1.02}
{'loss': 0.1656, 'learning_rate': 5.081156607209551e-06, 'epoch': 1.03}
{'loss': 0.2213, 'learning_rate': 5.074502143856309e-06, 'epoch': 1.03}
{'loss': 0.1683, 'learning_rate': 5.0678280626479585e-06, 'epoch': 1.03}
{'loss': 0.184, 'learning_rate': 5.0611344266989986e-06, 'epoch': 1.04}
{'loss': 0.1815, 'learning_rate': 5.0544212993088465e-06, 'epoch': 1.04}
{'loss': 0.2368, 'learning_rate': 5.047688743961246e-06, 'epoch': 1.05}
{'loss': 0.1766, 'learning_rate': 5.040936824323663e-06, 'epoch': 1.05}
{'loss': 0.2584, 'learning_rate': 5.034165604246685e-06, 'epoch

{'loss': 0.135, 'learning_rate': 4.435906374241082e-06, 'epoch': 1.37}
{'loss': 0.113, 'learning_rate': 4.42779947581233e-06, 'epoch': 1.37}
{'loss': 0.2132, 'learning_rate': 4.419679075173748e-06, 'epoch': 1.37}
{'loss': 0.2622, 'learning_rate': 4.411545249117182e-06, 'epoch': 1.38}
{'loss': 0.1331, 'learning_rate': 4.40339807456143e-06, 'epoch': 1.38}
{'loss': 0.1865, 'learning_rate': 4.395237628551529e-06, 'epoch': 1.39}
{'loss': 0.1622, 'learning_rate': 4.387063988258018e-06, 'epoch': 1.39}
{'loss': 0.1398, 'learning_rate': 4.378877230976209e-06, 'epoch': 1.39}
{'loss': 0.1681, 'learning_rate': 4.370677434125456e-06, 'epoch': 1.4}
{'loss': 0.306, 'learning_rate': 4.362464675248426e-06, 'epoch': 1.4}
{'loss': 0.1257, 'learning_rate': 4.3542390320103625e-06, 'epoch': 1.41}
{'loss': 0.3222, 'learning_rate': 4.346000582198352e-06, 'epoch': 1.41}
{'eval_loss': 0.2176375836133957, 'eval_content_rmse': 0.39037853479385376, 'eval_wording_rmse': 0.5318642854690552, 'eval_mcrmse': 0.46112141

...

{'loss': 0.0489, 'learning_rate': 1.9636398051945436e-06, 'epoch': 2.45}
{'loss': 0.1091, 'learning_rate': 1.9549871720627846e-06, 'epoch': 2.45}
{'loss': 0.1782, 'learning_rate': 1.947208249304401e-06, 'epoch': 2.46}
{'loss': 0.0843, 'learning_rate': 1.9385744624786323e-06, 'epoch': 2.46}
{'loss': 0.1025, 'learning_rate': 1.9299507131900454e-06, 'epoch': 2.47}
{'eval_loss': 0.24414509534835815, 'eval_content_rmse': 0.38082951307296753, 'eval_wording_rmse': 0.5858827233314514, 'eval_mcrmse': 0.4833561182022095, 'eval_runtime': 113.8625, 'eval_samples_per_second': 18.066, 'eval_steps_per_second': 9.037, 'epoch': 2.47}
{'loss': 0.1038, 'learning_rate': 1.9213370829904773e-06, 'epoch': 2.47}
{'loss': 0.1021, 'learning_rate': 1.912733653336076e-06, 'epoch': 2.47}
{'loss': 0.0982, 'learning_rate': 1.9041405055865235e-06, 'epoch': 2.48}
{'loss': 0.1185, 'learning_rate': 1.89555772100427e-06, 'epoch': 2.48}
{'loss': 0.0872, 'learning_rate': 1.8869853807537645e-06, 'epoch': 2.49}
{'loss': 0.10

{'loss': 0.2052, 'learning_rate': 1.2196707355829533e-06, 'epoch': 2.81}
{'loss': 0.0808, 'learning_rate': 1.2122537765135106e-06, 'epoch': 2.82}
{'loss': 0.082, 'learning_rate': 1.204853723546808e-06, 'epoch': 2.82}
{'eval_loss': 0.23207631707191467, 'eval_content_rmse': 0.37930914759635925, 'eval_wording_rmse': 0.5659306645393372, 'eval_mcrmse': 0.472619891166687, 'eval_runtime': 112.7493, 'eval_samples_per_second': 18.244, 'eval_steps_per_second': 9.126, 'epoch': 2.82}
{'loss': 0.0777, 'learning_rate': 1.1974706466626089e-06, 'epoch': 2.82}
{'loss': 0.041, 'learning_rate': 1.1901046156801392e-06, 'epoch': 2.83}
{'loss': 0.084, 'learning_rate': 1.1827557002574247e-06, 'epoch': 2.83}
{'loss': 0.0892, 'learning_rate': 1.1754239698906401e-06, 'epoch': 2.83}
{'loss': 0.0625, 'learning_rate': 1.1681094939134424e-06, 'epoch': 2.84}
{'loss': 0.0746, 'learning_rate': 1.1608123414963227e-06, 'epoch': 2.84}
{'loss': 0.0804, 'learning_rate': 1.1535325816459469e-06, 'epoch': 2.85}
{'loss': 0.091

{'eval_loss': 0.2535480558872223, 'eval_content_rmse': 0.38503673672676086, 'eval_wording_rmse': 0.5990350842475891, 'eval_mcrmse': 0.4920359253883362, 'eval_runtime': 113.832, 'eval_samples_per_second': 18.07, 'eval_steps_per_second': 9.04, 'epoch': 3.17}
{'loss': 0.0819, 'learning_rate': 6.107970230860707e-07, 'epoch': 3.18}
{'loss': 0.0457, 'learning_rate': 6.052290288075427e-07, 'epoch': 3.18}
{'loss': 0.0704, 'learning_rate': 5.99683681056553e-07, 'epoch': 3.18}
{'loss': 0.0707, 'learning_rate': 5.941610322735513e-07, 'epoch': 3.19}
{'loss': 0.0523, 'learning_rate': 5.886611346843337e-07, 'epoch': 3.19}
{'loss': 0.0459, 'learning_rate': 5.831840402995439e-07, 'epoch': 3.19}
{'loss': 0.0374, 'learning_rate': 5.77729800914186e-07, 'epoch': 3.2}
{'loss': 0.0473, 'learning_rate': 5.722984681071297e-07, 'epoch': 3.2}
{'loss': 0.0907, 'learning_rate': 5.66890093240625e-07, 'epoch': 3.21}
{'loss': 0.1264, 'learning_rate': 5.615047274598205e-07, 'epoch': 3.21}
{'loss': 0.092, 'learning_ra

{'loss': 0.061, 'learning_rate': 2.0596869153198018e-07, 'epoch': 3.53}
{'loss': 0.0481, 'learning_rate': 2.0262252111794355e-07, 'epoch': 3.53}
{'loss': 0.0902, 'learning_rate': 1.9930280454307014e-07, 'epoch': 3.54}
{'loss': 0.0474, 'learning_rate': 1.9600957320078316e-07, 'epoch': 3.54}
{'loss': 0.0541, 'learning_rate': 1.927428582340408e-07, 'epoch': 3.54}
{'loss': 0.0493, 'learning_rate': 1.8950269053504487e-07, 'epoch': 3.55}
{'loss': 0.0399, 'learning_rate': 1.8628910074495097e-07, 'epoch': 3.55}
{'loss': 0.0318, 'learning_rate': 1.8310211925357446e-07, 'epoch': 3.56}
{'loss': 0.0742, 'learning_rate': 1.799417761991069e-07, 'epoch': 3.56}
{'loss': 0.062, 'learning_rate': 1.7680810146782678e-07, 'epoch': 3.56}
{'loss': 0.0478, 'learning_rate': 1.7370112469382327e-07, 'epoch': 3.57}
{'loss': 0.1636, 'learning_rate': 1.706208752587085e-07, 'epoch': 3.57}
{'loss': 0.036, 'learning_rate': 1.6756738229134417e-07, 'epoch': 3.57}
{'loss': 0.0539, 'learning_rate': 1.6454067466756729e-07,

{'loss': 0.0445, 'learning_rate': 1.208657092324339e-08, 'epoch': 3.89}
{'loss': 0.0442, 'learning_rate': 1.127340906794816e-08, 'epoch': 3.89}
{'loss': 0.0625, 'learning_rate': 1.0488510574726661e-08, 'epoch': 3.9}
{'loss': 0.0734, 'learning_rate': 9.731882866095165e-09, 'epoch': 3.9}
{'loss': 0.0423, 'learning_rate': 9.003533097221794e-09, 'epoch': 3.9}
{'loss': 0.0488, 'learning_rate': 8.303468155861249e-09, 'epoch': 3.91}
{'loss': 0.0501, 'learning_rate': 7.63169466228686e-09, 'epoch': 3.91}
{'loss': 0.0671, 'learning_rate': 6.988218969230298e-09, 'epoch': 3.92}
{'loss': 0.0849, 'learning_rate': 6.3730471618203005e-09, 'epoch': 3.92}
{'loss': 0.0523, 'learning_rate': 5.786185057526039e-09, 'epoch': 3.92}
{'loss': 0.0876, 'learning_rate': 5.227638206100504e-09, 'epoch': 3.93}
{'loss': 0.0686, 'learning_rate': 4.6974118895302124e-09, 'epoch': 3.93}
{'loss': 0.0872, 'learning_rate': 4.195511121983575e-09, 'epoch': 3.94}
{'eval_loss': 0.24377551674842834, 'eval_content_rmse': 0.3834563

In [7]:
from pathlib import Path

seed = 42

fold = 3

output = f"output_fold{fold}_seed{seed}_1010"

!python train.py \
  --model_name_or_path "microsoft/deberta-v3-large" \
  --add_prompt_question True \
  --fold $fold \
  --data_dir "./" \
  --output_dir $output \
  --fp16 \
  --num_train_epochs 4 \
  --dataloader_num_workers 4 \
  --learning_rate 6e-6 \
  --weight_decay 1e-4 \
  --warmup_ratio 0 \
  --lr_scheduler_type "cosine" \
  --optim "adamw_torch" \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --evaluation_strategy "steps" \
  --eval_steps 150 \
  --save_strategy "steps" \
  --save_steps 150 \
  --save_total_limit 1 \
  --report_to "wandb" \
  --metric_for_best_model "mcrmse" \
  --greater_is_better False \
  --logging_steps 10 \
  --log_level "error" \
  --disable_tqdm True \
  --ddp_find_unused_parameters False \
  --dropout 0 \
  --seed $seed


output_dir = Path.cwd() / output
# add json files
for json_file in output_dir.glob("checkpoint*/*token*.json"):
    json_file.rename(output_dir/json_file.name)

# model files
for model_file in output_dir.glob("checkpoint*/*model*"):
    model_file.rename(output_dir/model_file.name)

# remove optimizer states and other files
to_delete = str(list(output_dir.glob("checkpoint*"))[0])
!rm -r $to_delete

wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_probs_dropout_prob": 0.0,
  "cfg": {
    "add_prompt_question": true,
    "add_prompt_text": false,
    "data_dir": "./",
    "dropout": 0.0,
    "fold": 3,
    "max_position_embeddings": 1600,
    "max_seq_length": 1600,
    "model_name_or_path": "microsoft/deberta-v3-large",
    "num_proc": 4
  },
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 1600,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
 

{'loss': 0.4436, 'learning_rate': 5.9273121952896085e-06, 'epoch': 0.28}
{'loss': 0.1399, 'learning_rate': 5.925304393836378e-06, 'epoch': 0.29}
{'loss': 0.5121, 'learning_rate': 5.923269588302444e-06, 'epoch': 0.29}
{'eval_loss': 0.5040857791900635, 'eval_content_rmse': 0.6781973838806152, 'eval_wording_rmse': 0.7404189109802246, 'eval_mcrmse': 0.7093081474304199, 'eval_runtime': 226.1087, 'eval_samples_per_second': 8.828, 'eval_steps_per_second': 4.414, 'epoch': 0.29}
{'loss': 0.2158, 'learning_rate': 5.921207797471509e-06, 'epoch': 0.29}
{'loss': 0.2072, 'learning_rate': 5.9191190403763836e-06, 'epoch': 0.3}
{'loss': 0.3666, 'learning_rate': 5.9170033362988126e-06, 'epoch': 0.3}
{'loss': 0.3505, 'learning_rate': 5.914860704769289e-06, 'epoch': 0.31}
{'loss': 0.4382, 'learning_rate': 5.912691165566883e-06, 'epoch': 0.31}
{'loss': 0.2009, 'learning_rate': 5.91049473871905e-06, 'epoch': 0.31}
{'loss': 0.3416, 'learning_rate': 5.908271444501456e-06, 'epoch': 0.32}
{'loss': 0.2545, 'lear

{'eval_loss': 0.2433973252773285, 'eval_content_rmse': 0.44721272587776184, 'eval_wording_rmse': 0.5355326533317566, 'eval_mcrmse': 0.4913727045059204, 'eval_runtime': 226.1497, 'eval_samples_per_second': 8.826, 'eval_steps_per_second': 4.413, 'epoch': 0.64}
{'loss': 0.1854, 'learning_rate': 5.628217149812086e-06, 'epoch': 0.64}
{'loss': 0.2644, 'learning_rate': 5.6238100201699425e-06, 'epoch': 0.65}
{'loss': 0.177, 'learning_rate': 5.619378669603068e-06, 'epoch': 0.65}
{'loss': 0.3495, 'learning_rate': 5.6149231390181625e-06, 'epoch': 0.65}
{'loss': 0.1379, 'learning_rate': 5.610443469545135e-06, 'epoch': 0.66}
{'loss': 0.1975, 'learning_rate': 5.605939702536728e-06, 'epoch': 0.66}
{'loss': 0.2505, 'learning_rate': 5.601411879568132e-06, 'epoch': 0.67}
{'loss': 0.2167, 'learning_rate': 5.596860042436603e-06, 'epoch': 0.67}
{'loss': 0.2355, 'learning_rate': 5.592284233161077e-06, 'epoch': 0.67}
{'loss': 0.144, 'learning_rate': 5.587684493981782e-06, 'epoch': 0.68}
{'loss': 0.265, 'lear

{'loss': 0.2134, 'learning_rate': 5.13992880368732e-06, 'epoch': 0.99}
{'loss': 0.2844, 'learning_rate': 5.133530786719845e-06, 'epoch': 0.99}
{'loss': 0.2605, 'learning_rate': 5.127113074695106e-06, 'epoch': 1.0}
{'loss': 0.3182, 'learning_rate': 5.120675726856309e-06, 'epoch': 1.0}
{'loss': 0.2092, 'learning_rate': 5.114218802627925e-06, 'epoch': 1.01}
{'loss': 0.3251, 'learning_rate': 5.1077423616151366e-06, 'epoch': 1.01}
{'loss': 0.1929, 'learning_rate': 5.101246463603289e-06, 'epoch': 1.01}
{'loss': 0.204, 'learning_rate': 5.09473116855734e-06, 'epoch': 1.02}
{'loss': 0.1294, 'learning_rate': 5.088196536621306e-06, 'epoch': 1.02}
{'loss': 0.2388, 'learning_rate': 5.0816426281177035e-06, 'epoch': 1.03}
{'loss': 0.0769, 'learning_rate': 5.075069503547e-06, 'epoch': 1.03}
{'loss': 0.1542, 'learning_rate': 5.068477223587044e-06, 'epoch': 1.03}
{'loss': 0.2262, 'learning_rate': 5.061865849092516e-06, 'epoch': 1.04}
{'loss': 0.2427, 'learning_rate': 5.055235441094361e-06, 'epoch': 1.04

{'loss': 0.2042, 'learning_rate': 4.4688450310316255e-06, 'epoch': 1.35}
{'loss': 0.1801, 'learning_rate': 4.460890653513704e-06, 'epoch': 1.35}
{'loss': 0.1178, 'learning_rate': 4.452922790216595e-06, 'epoch': 1.36}
{'loss': 0.1426, 'learning_rate': 4.4449415146932685e-06, 'epoch': 1.36}
{'loss': 0.2437, 'learning_rate': 4.436946900620505e-06, 'epoch': 1.37}
{'loss': 0.1299, 'learning_rate': 4.428939021798215e-06, 'epoch': 1.37}
{'loss': 0.1899, 'learning_rate': 4.420917952148759e-06, 'epoch': 1.37}
{'loss': 0.1251, 'learning_rate': 4.4128837657162655e-06, 'epoch': 1.38}
{'loss': 0.1972, 'learning_rate': 4.404836536665945e-06, 'epoch': 1.38}
{'loss': 0.3737, 'learning_rate': 4.3967763392834086e-06, 'epoch': 1.38}
{'loss': 0.1118, 'learning_rate': 4.388703247973978e-06, 'epoch': 1.39}
{'loss': 0.2158, 'learning_rate': 4.380617337262006e-06, 'epoch': 1.39}
{'eval_loss': 0.21478497982025146, 'eval_content_rmse': 0.4156012535095215, 'eval_wording_rmse': 0.5067992806434631, 'eval_mcrmse': 

...

{'loss': 0.094, 'learning_rate': 2.028851408684911e-06, 'epoch': 2.42}
{'loss': 0.0923, 'learning_rate': 2.020231829419156e-06, 'epoch': 2.43}
{'loss': 0.0767, 'learning_rate': 2.0116212945929488e-06, 'epoch': 2.43}
{'loss': 0.1067, 'learning_rate': 2.0030198836918903e-06, 'epoch': 2.43}
{'loss': 0.1593, 'learning_rate': 1.9944276761173576e-06, 'epoch': 2.44}
{'eval_loss': 0.22620804607868195, 'eval_content_rmse': 0.41844868659973145, 'eval_wording_rmse': 0.5266090035438538, 'eval_mcrmse': 0.4725288450717926, 'eval_runtime': 226.0841, 'eval_samples_per_second': 8.829, 'eval_steps_per_second': 4.414, 'epoch': 2.44}
{'loss': 0.1415, 'learning_rate': 1.9858447511857674e-06, 'epoch': 2.44}
{'loss': 0.1036, 'learning_rate': 1.97727118812785e-06, 'epoch': 2.44}
{'loss': 0.1127, 'learning_rate': 1.9687070660879113e-06, 'epoch': 2.45}
{'loss': 0.1238, 'learning_rate': 1.960152464123104e-06, 'epoch': 2.45}
{'loss': 0.0663, 'learning_rate': 1.951607461202705e-06, 'epoch': 2.46}
{'loss': 0.1406, 

{'loss': 0.14, 'learning_rate': 1.2834211770973932e-06, 'epoch': 2.78}
{'loss': 0.0659, 'learning_rate': 1.275953831328445e-06, 'epoch': 2.78}
{'loss': 0.0741, 'learning_rate': 1.2685024005808446e-06, 'epoch': 2.79}
{'eval_loss': 0.21390452980995178, 'eval_content_rmse': 0.39798176288604736, 'eval_wording_rmse': 0.5190566182136536, 'eval_mcrmse': 0.45851919054985046, 'eval_runtime': 225.9373, 'eval_samples_per_second': 8.834, 'eval_steps_per_second': 4.417, 'epoch': 2.79}
{'loss': 0.0637, 'learning_rate': 1.2610669536402673e-06, 'epoch': 2.79}
{'loss': 0.084, 'learning_rate': 1.2536475591448399e-06, 'epoch': 2.79}
{'loss': 0.1471, 'learning_rate': 1.2462442855845009e-06, 'epoch': 2.8}
{'loss': 0.1157, 'learning_rate': 1.2388572013003784e-06, 'epoch': 2.8}
{'loss': 0.1275, 'learning_rate': 1.231486374484153e-06, 'epoch': 2.8}
{'loss': 0.1235, 'learning_rate': 1.2241318731774248e-06, 'epoch': 2.81}
{'loss': 0.1158, 'learning_rate': 1.216793765271098e-06, 'epoch': 2.81}
{'loss': 0.1133, '

{'loss': 0.0972, 'learning_rate': 6.712413399053479e-07, 'epoch': 3.13}
{'eval_loss': 0.20719866454601288, 'eval_content_rmse': 0.39978092908859253, 'eval_wording_rmse': 0.5045515894889832, 'eval_mcrmse': 0.45216625928878784, 'eval_runtime': 226.1572, 'eval_samples_per_second': 8.826, 'eval_steps_per_second': 4.413, 'epoch': 3.13}
{'loss': 0.0476, 'learning_rate': 6.655058845388638e-07, 'epoch': 3.14}
{'loss': 0.0386, 'learning_rate': 6.597919793630647e-07, 'epoch': 3.14}
{'loss': 0.0425, 'learning_rate': 6.540996771241739e-07, 'epoch': 3.15}
{'loss': 0.0282, 'learning_rate': 6.484290303689903e-07, 'epoch': 3.15}
{'loss': 0.0998, 'learning_rate': 6.427800914444125e-07, 'epoch': 3.15}
{'loss': 0.0714, 'learning_rate': 6.371529124969437e-07, 'epoch': 3.16}
{'loss': 0.0841, 'learning_rate': 6.3154754547222e-07, 'epoch': 3.16}
{'loss': 0.0541, 'learning_rate': 6.259640421145268e-07, 'epoch': 3.16}
{'loss': 0.0528, 'learning_rate': 6.204024539663191e-07, 'epoch': 3.17}
{'loss': 0.079, 'lear

{'loss': 0.0417, 'learning_rate': 2.434165271881599e-07, 'epoch': 3.49}
{'loss': 0.0484, 'learning_rate': 2.3983270049889494e-07, 'epoch': 3.49}
{'loss': 0.031, 'learning_rate': 2.3627435347618098e-07, 'epoch': 3.49}
{'loss': 0.0801, 'learning_rate': 2.3274151896784412e-07, 'epoch': 3.5}
{'loss': 0.0539, 'learning_rate': 2.2923422958619856e-07, 'epoch': 3.5}
{'loss': 0.0761, 'learning_rate': 2.2575251770774774e-07, 'epoch': 3.5}
{'loss': 0.097, 'learning_rate': 2.222964154728827e-07, 'epoch': 3.51}
{'loss': 0.0371, 'learning_rate': 2.188659547855879e-07, 'epoch': 3.51}
{'loss': 0.051, 'learning_rate': 2.1546116731314313e-07, 'epoch': 3.52}
{'loss': 0.1002, 'learning_rate': 2.120820844858371e-07, 'epoch': 3.52}
{'loss': 0.074, 'learning_rate': 2.0872873749667033e-07, 'epoch': 3.52}
{'loss': 0.0757, 'learning_rate': 2.05401157301072e-07, 'epoch': 3.53}
{'loss': 0.0525, 'learning_rate': 2.0209937461661298e-07, 'epoch': 3.53}
{'loss': 0.0791, 'learning_rate': 1.988234199227219e-07, 'epoch'

{'loss': 0.0766, 'learning_rate': 2.393086246032372e-08, 'epoch': 3.84}
{'loss': 0.0358, 'learning_rate': 2.279561040103384e-08, 'epoch': 3.85}
{'loss': 0.0432, 'learning_rate': 2.1687841521880968e-08, 'epoch': 3.85}
{'loss': 0.0622, 'learning_rate': 2.0607566048905124e-08, 'epoch': 3.85}
{'loss': 0.0446, 'learning_rate': 1.9554793954349337e-08, 'epoch': 3.86}
{'loss': 0.0447, 'learning_rate': 1.8529534956567395e-08, 'epoch': 3.86}
{'loss': 0.0628, 'learning_rate': 1.7531798519933585e-08, 'epoch': 3.86}
{'loss': 0.0597, 'learning_rate': 1.6561593854756752e-08, 'epoch': 3.87}
{'loss': 0.0773, 'learning_rate': 1.5618929917193047e-08, 'epoch': 3.87}
{'loss': 0.0489, 'learning_rate': 1.4703815409165655e-08, 'epoch': 3.88}
{'loss': 0.072, 'learning_rate': 1.3816258778283186e-08, 'epoch': 3.88}
{'loss': 0.0319, 'learning_rate': 1.2956268217761414e-08, 'epoch': 3.88}
{'loss': 0.0255, 'learning_rate': 1.212385166634833e-08, 'epoch': 3.89}
{'eval_loss': 0.20844817161560059, 'eval_content_rmse':

In [10]:
from pathlib import Path
import json

seed = 42
scores = []
for fold in range(0,4):
    output = f"/root/autodl-tmp/output_fold{fold}_seed{seed}_0810"
    p = Path(output) / "config.json"

    with open(p) as fp:
        cfg = json.load(fp)


    print(f"MCRMSE for fold {fold}, seed {seed}: {cfg['best_metric']:.4f}")
    scores.append(cfg['best_metric'])
print(sum(scores)/len(scores))

MCRMSE for fold 0, seed 42: 0.5363
MCRMSE for fold 1, seed 42: 0.4454
MCRMSE for fold 2, seed 42: 0.4974
MCRMSE for fold 3, seed 42: 0.4314
0.47761109471321106


#df.loc[0, 'prompt_text']

In [18]:
# from pathlib import Path
# import json

# seed = 42
# scores = []
# for fold in range(4):
#     output = f"./output_fold{fold}_seed{seed}_3008"
#     p = Path(output) / "config.json"

#     with open(p) as fp:
#         cfg = json.load(fp)


#     print(f"MCRMSE for fold {fold}, seed {seed}: {cfg['best_metric']:.4f}")
#     scores.append(cfg['best_metric'])
# print(sum(scores)/len(scores))